# Urban PointCloud Processing

This notebook shows a "complete solution" in which a single point cloud file is automatically labeled using the various tools available in this repository. For clarity we skip [preprocessing of AHN data](1.%20AHN%20preprocessing.ipynb) and assume all necassary data files are already available.

We build a `Pipeline` with different `Processors`, each of which labels a particular type of object. The result is a labelled pointcloud, where labels are stored in the LAS extra_dim `label`.

The `Pipeline` supports processing a single file, or batch-processing a folder.

In [ ]:
# Add project src to path.
import set_path

# Import modules.
import logging

import src.fusion as fusion
import src.region_growing as growing
import src.utils.ahn_utils as ahn_utils
import src.utils.plot_utils as plot_utils
import src.utils.log_utils as log_utils
from src.pipeline import Pipeline
from src.utils.labels import Labels

In [ ]:
# Set-up.

# AHN data folder.
ahn_data_folder = '../datasets/ahn/'
# File with BGT road polygons.
bgt_road_file = '../datasets/bgt/bgt_roads_demo.csv'
# File with BGT building polygons.
bgt_building_file = '../datasets/bgt/bgt_buildings_demo.csv'
# File with <x,y> coordinates of pole-like objects.
bgt_point_file = '../datasets/bgt/custom_points_demo.csv'

# Some methods use AHN elevation data to determine the height above ground.
ahn_reader = ahn_utils.NPZReader(ahn_data_folder)

In [ ]:
# Set-up logging.
logfile = '../datasets/pointcloud/pipeline.log'
log_utils.reset_logger()

# INFO messages will be printed to console.
log_utils.add_console_logger()
# All messages will be printed to a file. Use `clear_log` flag to clear the log file, if desired.
log_utils.add_file_logger(logfile, clear_log=True)

---
## Data Fusion

### Ground and Buildings
First, we use data fusion to automatically label ground and building points.

For details and more options, see notebook [2. Ground and Buildings](2.%20Ground%20and%20Buildings.ipynb).

In [ ]:
# Ground fuser using pre-processed AHN data.
npz_ground_fuser = fusion.AHNFuser(Labels.GROUND, ahn_data_folder,
                                   method='npz', target='ground', epsilon=0.2)
# Building fuser using BGT building footprint data and AHN data for the maximum height, if available.
bgt_building_fuser = fusion.BGTBuildingFuser(Labels.BUILDING, building_offset=0.25, bgt_file=bgt_building_file, ahn_reader=ahn_reader)

### Noise filter
Before actually performing the building fusion, we detect and remove outliers and noise (i.e. wrong measurements) from the point cloud.

In [ ]:
# Noise filter using a clustering algorithm and AHN elevation data.
noise_filter = fusion.NoiseFilter(Labels.NOISE, ahn_reader, epsilon=0.2, min_component_size=100)

### Cars

Then we label cars by searching for 'car-like' clusters above road segments.

For details and on how to generate the BGT road csv file, see notebook [3. Cars](3.%20Cars.ipynb).

In [ ]:
# Car fuser using a clustering algorithm and BGT road data.
car_fuser = fusion.CarFuser(Labels.CAR, ahn_reader, bgt_file=bgt_road_file,
                            octree_level=10, min_component_size=5000,
                            min_height=1.2, max_height=2.4,
                            min_width=1.4, max_width=2.4,
                            min_length=3.0, max_length=6.0)

### Pole-like objects

Finally we look for 'pole-like' objects such as trees, street lights and traffic signs.

For details and more options, see notebook [4. Pole-like objects](4.%20Pole-like%20objects.ipynb).

In [ ]:
# Parameter settings for each type of object.
tree_params = {'seed_height': 1.75, 'min_points': 500, 'max_r': 0.5, 'label_height': 5.}
light_params = {'seed_height': 2.25, 'min_points': 400, 'max_r': 0.2, 'label_height': 5.}
sign_params = {'seed_height': 1.75, 'min_points': 200, 'max_r': 0.2, 'min_height': 1.2, 'z_max': 2., 'label_height': 3.}

# Fusers for BGT point data.
tree_fuser = fusion.BGTPointFuser(Labels.TREE, bgt_type='boom', bgt_file=bgt_point_file, ahn_reader=ahn_reader, params=tree_params)
light_fuser = fusion.BGTPointFuser(Labels.STREET_LIGHT, bgt_type='lichtmast', bgt_file=bgt_point_file, ahn_reader=ahn_reader, params=light_params)
sign_fuser = fusion.BGTPointFuser(Labels.TRAFFIC_SIGN, bgt_type='verkeersbord', bgt_file=bgt_point_file, ahn_reader=ahn_reader, params=sign_params)

---
## Region Growing

We use region growing based on labelling of connected components. For details and other options see notebook [5. Region growing](5.%20Region%20growing.ipynb).

### Buildings

We use region growing to refine the buildings. This will make sure that protruding elements such as balconies are labelled correctly.

In [ ]:
# We do this separately for the top and bottom since the best settings for each might differ.
building_top = {'bottom': 12., 'octree_level': 9, 'threshold': 0.5}
building_bottom = {'bottom': 0.5, 'top': 12., 'octree_level': 10, 'threshold': 0.5}

building_grower = growing.LayerLCC(Labels.BUILDING, ahn_reader, 
                                   params=[building_top, building_bottom])

### Pole-like objects

We also use region growing to refine the pole-like objects. We do this separately for the top and bottom since the best settings for each might differ.

For details and more options, see notebook [4. Pole-like objects](4.%20Pole-like%20objects.ipynb).

In [ ]:
light_bottom = {'top': 1.75, 'octree_level': 10, 'threshold': 0.5}
light_top = {'bottom': 1.75, 'octree_level': 9, 'threshold': 0.05}

sign_bottom = {'top': 1.5, 'octree_level': 10, 'threshold': 0.5}
sign_top = {'bottom': 1.5, 'octree_level': 9, 'threshold': 0.05}

tree_bottom = {'top': 1.75, 'octree_level': 10, 'threshold': 0.8}
tree_middle = {'bottom': 1.75, 'top':10.0, 'octree_level': 9, 'threshold': 0.01}
tree_top = {'bottom': 6.0, 'octree_level': 8, 'threshold': 0.01}

light_grower = growing.LayerLCC(Labels.STREET_LIGHT, ahn_reader, 
                                params=[light_bottom, light_top])
sign_grower = growing.LayerLCC(Labels.TRAFFIC_SIGN, ahn_reader, 
                               params=[sign_bottom, sign_top])
tree_grower = growing.LayerLCC(Labels.TREE, ahn_reader, reset_noise=True,
                               params=[tree_bottom, tree_middle, tree_top])

---
## Set-up the pipeline

In [ ]:
# Set-up pipeline.
process_sequence = (npz_ground_fuser, noise_filter, bgt_building_fuser, car_fuser, tree_fuser,
                    light_fuser, sign_fuser, building_grower, light_grower, sign_grower, tree_grower)
pipeline = Pipeline(process_sequence)

---
## Process a single file

In [ ]:
# Select the file to process.
tile_code = '2386_9702'  # Demo tile 1.
#tile_code = '2397_9705'  # Demo tile 2.

in_file = '../datasets/pointcloud/filtered_' + tile_code + '.laz'
out_file = '../datasets/pointcloud/processed_' + tile_code + '.laz'

In [ ]:
pipeline.process_file(in_file, out_file)

---
## Process a folder

It is also possible to process all LAS files in a folder in one batch.

In [ ]:
# Select the folder to process.
las_folder = '../datasets/pointcloud/'
out_folder = '../datasets/pointcloud/'
# Only files starting with the specified prefix will be processed.
in_prefix = 'filtered_'
# The prefix will be replaced for processed files (otherwise the input might be overwritten).
out_prefix = 'processed_'

# Re-set logging to suppress console output.
log_utils.reset_logger()

# Only ERROR messages will be printed to console.
log_utils.add_console_logger(level=logging.ERROR)
# All messages will be printed to a file. Use `clear_log` flag to clear the log file, if desired.
log_utils.add_file_logger(logfile, clear_log=True)

In [ ]:
pipeline.process_folder(in_folder=las_folder, out_folder=out_folder, in_prefix=in_prefix, out_prefix=out_prefix)

---
## View the result

To get a quick look at the result, we can plot a slice of the labelled point cloud at a specified height.

In [ ]:
%matplotlib widget
plot_utils.plot_cloud_slice(out_file, ahn_reader, plane_height=1.4)

### In CloudCompare
The final result can also be viewed in CloudCompare. Open the out_file (e.g. datasets/pointcloud/labelled_2386_9702.laz) and change `Colors` from "RGB" to "Scalar field". For best results, use our [custom color scale](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/raw/main/media/cc_color_scale.xml), and set "steps" to 100.

The result should look like this:
![Demo result](https://github.com/Amsterdam-AI-Team/Urban_PointCloud_Processing/raw/main/media/examples/demo_result.png)